# <center> Selection cohort of patients and linked x_ray images

#### **1/ Import all necessary libraries and classes to run this notebook**

In [ ]:
from datetime import timedelta
import os

import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from IPython.display import display, HTML, Image
%matplotlib inline

plt.style.use('ggplot')
plt.rcParams.update({'font.size': 20})

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

#### **2/ Set up MIMIC big query environment**

In [ ]:
# authenticate
auth.authenticate_user()

In [ ]:
# Set up environment variables
project_id = 'your_project_id'
os.environ["GOOGLE_CLOUD_PROJECT"] = project_id

# Read data from BigQuery into pandas dataframes.
def run_query(query, project_id=project_id):
  return pd.io.gbq.read_gbq(
      query,
      project_id=project_id,
      dialect='standard')

# set the dataset
dataset = 'mimiciv'


#### **3/ Query the x_rays of our cohort of patients**

In [ ]:
# Find ICD code related to pneumonia, tuberculosis and lung cancer

df = run_query("""
SELECT icd_code, long_title FROM `physionet-data.mimiciv_hosp.d_icd_diagnoses`
WHERE LOWER(long_title) LIKE "%pneumonia%" OR LOWER(long_title) LIKE "%tuberculosis%" OR LOWER(long_title) LIKE "%bronchitis%"
""")

# ICD code list
icd_codes = df['icd_code'].to_list()
icd_codes_string = ', '.join(["'" + icd + "'" for icd in icd_codes])
# Subject ID that have an ICD code related to pneumonia, tuberculosis and bronchitis and a CHEST x-ray

all_subject_ids = run_query(f"""
SELECT `physionet-data.mimiciv_hosp.patients`.subject_id
FROM `physionet-data.mimiciv_hosp.patients`
JOIN `physionet-data.mimiciv_hosp.admissions` ON `physionet-data.mimiciv_hosp.patients`.subject_id = `physionet-data.mimiciv_hosp.admissions`.subject_id
JOIN `physionet-data.mimiciv_hosp.diagnoses_icd` ON `physionet-data.mimiciv_hosp.admissions`.hadm_id = `physionet-data.mimiciv_hosp.diagnoses_icd`.hadm_id
JOIN `physionet-data.mimic_cxr.record_list` ON `physionet-data.mimiciv_hosp.patients`.subject_id = `physionet-data.mimic_cxr.record_list`.subject_id
JOIN `physionet-data.mimic_cxr.dicom_metadata_string` ON `physionet-data.mimic_cxr.record_list`.dicom_id = `physionet-data.mimic_cxr.dicom_metadata_string`.dicom
WHERE `physionet-data.mimiciv_hosp.diagnoses_icd`.icd_code IN ({icd_codes_string}) AND `physionet-data.mimic_cxr.dicom_metadata_string`.BodyPartExamined = 'CHEST'
GROUP BY `physionet-data.mimiciv_hosp.patients`.subject_id
""")
print(all_subject_ids)


       subject_id
0        10361837
1        10433099
2        12481952
3        12862321
4        16662264
...           ...
11329    18505859
11330    10940509
11331    11080025
11332    13588348
11333    14381700

[11334 rows x 1 columns]


In [ ]:
subject_hadm = run_query(f"""
SELECT
    `physionet-data.mimiciv_hosp.patients`.subject_id,
    ARRAY_AGG(DISTINCT `physionet-data.mimiciv_hosp.admissions`.hadm_id) AS hadm_ids
FROM
    `physionet-data.mimiciv_hosp.patients`
JOIN
    `physionet-data.mimiciv_hosp.admissions` ON `physionet-data.mimiciv_hosp.patients`.subject_id = `physionet-data.mimiciv_hosp.admissions`.subject_id
JOIN
    `physionet-data.mimiciv_hosp.diagnoses_icd` ON `physionet-data.mimiciv_hosp.admissions`.hadm_id = `physionet-data.mimiciv_hosp.diagnoses_icd`.hadm_id
JOIN
    `physionet-data.mimic_cxr.record_list` ON `physionet-data.mimiciv_hosp.patients`.subject_id = `physionet-data.mimic_cxr.record_list`.subject_id
JOIN
    `physionet-data.mimic_cxr.dicom_metadata_string` ON `physionet-data.mimic_cxr.record_list`.dicom_id = `physionet-data.mimic_cxr.dicom_metadata_string`.dicom
WHERE
    `physionet-data.mimiciv_hosp.diagnoses_icd`.icd_code IN ({icd_codes_string})
AND
    `physionet-data.mimic_cxr.dicom_metadata_string`.BodyPartExamined = 'CHEST'
GROUP BY
    `physionet-data.mimiciv_hosp.patients`.subject_id

""")
subject_hadm

,subject_id,hadm_ids
0,10361837,"[23346424, 22076746, 27515984]"
1,10433099,[28593361]
2,12481952,"[24985536, 27017213, 27857372, 22058685, 20146..."
3,12862321,"[27343985, 22439772, 26167368]"
4,16662264,"[22174937, 26390489, 21137571, 21588032, 28552..."
...,...,...
11329,18505859,[20886432]
11330,10940509,[26533493]
11331,11080025,[25064210]
11332,13588348,[29652751]


In [ ]:
subject_hadm_icd = run_query(f"""
SELECT
    `physionet-data.mimiciv_hosp.patients`.subject_id,
    `physionet-data.mimiciv_hosp.admissions`.hadm_id,
    ARRAY_AGG(DISTINCT `physionet-data.mimiciv_hosp.diagnoses_icd`.icd_code) AS icd_codes
FROM
    `physionet-data.mimiciv_hosp.patients`
JOIN
    `physionet-data.mimiciv_hosp.admissions` ON `physionet-data.mimiciv_hosp.patients`.subject_id = `physionet-data.mimiciv_hosp.admissions`.subject_id
JOIN
    `physionet-data.mimiciv_hosp.diagnoses_icd` ON `physionet-data.mimiciv_hosp.admissions`.hadm_id = `physionet-data.mimiciv_hosp.diagnoses_icd`.hadm_id
JOIN
    `physionet-data.mimic_cxr.record_list` ON `physionet-data.mimiciv_hosp.patients`.subject_id = `physionet-data.mimic_cxr.record_list`.subject_id
JOIN
    `physionet-data.mimic_cxr.dicom_metadata_string` ON `physionet-data.mimic_cxr.record_list`.dicom_id = `physionet-data.mimic_cxr.dicom_metadata_string`.dicom
WHERE
    `physionet-data.mimiciv_hosp.diagnoses_icd`.icd_code IN ({icd_codes_string})
AND
    `physionet-data.mimic_cxr.dicom_metadata_string`.BodyPartExamined = 'CHEST'
GROUP BY
    `physionet-data.mimiciv_hosp.patients`.subject_id, `physionet-data.mimiciv_hosp.admissions`.hadm_id
""")
subject_hadm_icd

,subject_id,hadm_id,icd_codes
0,10433099,28593361,"[48283, 99731]"
1,12481952,24985536,[4870]
2,14020056,27810404,[4870]
3,19817306,21573368,[4847]
4,13624277,22084051,[4847]
...,...,...,...
18631,17190208,26495862,[J95851]
18632,11064691,23704085,[J95851]
18633,17640354,20483724,[J95851]
18634,13349882,28304816,[J95851]


In [ ]:
subject_hadm_icd_dicom = run_query(f"""
WITH unique_dicoms AS (
    SELECT DISTINCT
        rl.subject_id,
        rl.dicom_id
    FROM
        `physionet-data.mimic_cxr.record_list` rl
    JOIN
        `physionet-data.mimic_cxr.dicom_metadata_string` dms ON rl.dicom_id = dms.dicom
    WHERE
        dms.BodyPartExamined = 'CHEST'
    AND
        dms.ViewPosition IN ('AP', 'PA', 'LATERAL', 'LL')
)

SELECT
    p.subject_id,
    a.hadm_id,
    ARRAY_AGG(DISTINCT di.icd_code) AS icd_codes,
    ARRAY_AGG(DISTINCT ud.dicom_id) AS dicom_ids
FROM
    `physionet-data.mimiciv_hosp.patients` p
JOIN
    `physionet-data.mimiciv_hosp.admissions` a ON p.subject_id = a.subject_id
JOIN
    `physionet-data.mimiciv_hosp.diagnoses_icd` di ON a.hadm_id = di.hadm_id
JOIN
    unique_dicoms ud ON p.subject_id = ud.subject_id
WHERE
    di.icd_code IN ({icd_codes_string})
GROUP BY
    p.subject_id, a.hadm_id

""")
subject_hadm_icd_dicom

,subject_id,hadm_id,icd_codes,dicom_ids
0,18415616,21610375,"[4870, 4829]","[75d67482-46fbfcfb-b9d3be10-98f1b1dd-ba9748dc,..."
1,10554112,26434926,[J208],"[c0c75f58-d72e248e-3235d33d-61afd4b9-cf364a7f,..."
2,10951680,27317933,[7955],"[8cd297c0-26db47a0-38912029-b8f6dc6f-7fff6c86,..."
3,13056000,26373796,"[J189, R7611]","[81850da3-d56e0868-2340e5c6-ec7adc74-dca4eb9e,..."
4,13450240,26133718,"[V066, V1201]","[a7fc8c47-b8e8d3ef-c62c2a6d-29fc23f6-4311cc42,..."
...,...,...,...,...
18481,11277562,29036184,[J95851],"[7d8d88b7-95a89f82-4995a922-5ecc9ca2-72d19481,..."
18482,13349882,28304816,[J95851],"[6914cdd8-3f14cef0-fc2f11da-2a153aab-3e238b75,..."
18483,17399295,26020471,[J95851],"[a835135c-da5a2cbf-bd448e1c-f2fd438a-95e0d387,..."
18484,17399295,28061202,[J95851],"[a835135c-da5a2cbf-bd448e1c-f2fd438a-95e0d387,..."


In [ ]:
dicom_mono = run_query(f"""
SELECT
    dicom,
    PhotometricInterpretation
FROM
    `physionet-data.mimic_cxr.dicom_metadata_string`
WHERE
    PhotometricInterpretation IN ('MONOCHROME1', 'MONOCHROME2')
""")
dicom_mono

,dicom,PhotometricInterpretation
0,7c8c895f-9b2c9cd3-2f8f7883-82ac6742-03022069,MONOCHROME2
1,55cd1e28-01d2cf86-d22f1ed9-b0379f91-b5208ea1,MONOCHROME2
2,f0256265-5d4d2a8b-5823ad83-e3aa1eb5-1b5501e2,MONOCHROME2
3,554d9307-473a1a17-5f770fb7-f33bc944-d8f6fdc8,MONOCHROME2
4,9bddc1c3-1566be3b-32232bb6-14734bc9-1478e217,MONOCHROME2
...,...,...
377105,a8443aa9-f95a678c-d2d31033-f90aa5ba-e0cccef5,MONOCHROME2
377106,ebd49d88-3299e60a-7876e5df-cedbc357-0962f7bf,MONOCHROME2
377107,2beacbe0-0c973e5a-dfdef725-f9086583-0db554ad,MONOCHROME2
377108,b738506c-fa73ec3f-12b429ad-2484eeae-93d6920d,MONOCHROME2


In [ ]:
# Expand the 'dicom_ids' column to multiple rows
s = subject_hadm_icd_dicom.apply(lambda x: pd.Series(x['dicom_ids']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'dicom'
expanded_df = subject_hadm_icd_dicom.drop('dicom_ids', axis=1).join(s)

# Merge with the dicom_mono dataframe
merged_df = pd.merge(expanded_df, dicom_mono, on='dicom', how='left')
merged_df


,subject_id,hadm_id,icd_codes,dicom,PhotometricInterpretation
0,18415616,21610375,"[4870, 4829]",75d67482-46fbfcfb-b9d3be10-98f1b1dd-ba9748dc,MONOCHROME2
1,18415616,21610375,"[4870, 4829]",91ea24c1-ddf8f918-0c579885-c0bf36ed-3a2b306a,MONOCHROME2
2,10554112,26434926,[J208],c0c75f58-d72e248e-3235d33d-61afd4b9-cf364a7f,MONOCHROME2
3,10554112,26434926,[J208],776eea09-08ed17b4-7c66b223-539bcd2e-f632edbd,MONOCHROME2
4,10554112,26434926,[J208],ad0ffda0-55795145-ed32a1d2-06aeaa05-47de5964,MONOCHROME2
...,...,...,...,...,...
297117,17399295,25950154,[J95851],70584b48-9ca2f049-2b48d576-1b9587d1-963ba7dc,MONOCHROME2
297118,17399295,25950154,[J95851],e15e563b-fc7da9ee-9abf32ae-cbfefc68-1e564904,MONOCHROME2
297119,17399295,25950154,[J95851],4e9a6961-77118031-4ec77ca2-5a61b1fa-0f49336e,MONOCHROME2
297120,17399295,25950154,[J95851],a8fa26ea-9f8bb5e0-c8df13e9-d330d83f-8c3aff8c,MONOCHROME2


In [ ]:
# Filter rows with MONOCHROME1
mono1_df = merged_df[merged_df['PhotometricInterpretation'] == 'MONOCHROME1']

# Filter rows with MONOCHROME2
mono2_df = merged_df[merged_df['PhotometricInterpretation'] == 'MONOCHROME2']

mono1_dicoms_unique = mono1_df['dicom'].unique().tolist()
mono2_dicoms_unique = mono2_df['dicom'].unique().tolist()

len(mono2_dicoms_unique)

123734

In [ ]:
# Step 1: Expand the dicom_ids column
expanded_df = subject_hadm_icd_dicom.explode('dicom_ids')

# Step 2: Merge with dicom_mono on the dicom column
merged_df = expanded_df.merge(dicom_mono[dicom_mono['PhotometricInterpretation'] == 'MONOCHROME2'],
                              left_on='dicom_ids', right_on='dicom', how='inner')

# Step 3: Select the desired columns
result = merged_df[['dicom', 'icd_codes']]

# Group by 'dicom' and aggregate 'icd_codes' as a single list
result = result.groupby('dicom')['icd_codes'].agg(lambda x: [code for codes in x for code in codes]).reset_index()

result


,dicom,icd_codes
0,00000218-9fb20d4e-86045713-8013e08b-0d5bebba,[J189]
1,0000d3be-591ae3b7-b03a7497-8319c02b-650bb4ab,[V0382]
2,0001bcde-bb754f26-6b32a474-90653a5d-3730aa89,"[4829, 49121, 49121, V1261, 4821, 48242, 49121..."
3,0002c540-fbe0f7cc-48725501-5ab948f7-8af53bbf,"[4820, 4821]"
4,0003494b-20c32dda-59f01833-783fbd4b-8d48e435,[J95851]
...,...,...
123729,fffd4d33-422d4d5b-7c09efc3-43832cf6-87c52a7f,"[J189, V0382]"
123730,fffea180-bc36450c-fb6d824a-4d089ba8-136b18c5,"[486, 486]"
123731,ffff5313-458549ff-8ef54533-add9597f-a0db64b7,"[79552, 7955, 7955, 486, 486, 486, 4829, 49122..."
123732,ffffc036-cbe84497-db8882f4-6bc3eada-fc5ca53f,"[5060, 5060, 49121, 49121, V1261, 5060, 5060, ..."
